# Features Extractor

This notebook demonstrates how to extract features from images using a pretrained ResNet50 model for dimensionality reduction techniques.

**Note:** This notebook extracts deep learning features from cat and dog images using a pretrained ResNet50 model and saves them for use in dimensionality reduction analysis.

## Import Required Libraries

Import the necessary libraries including torch, torchvision, PIL, and numpy.

In [1]:
# Import necessary libraries.
from typing import Tuple

import os
import numpy as np
import torch

import torchvision.transforms as transforms

from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose

from torchvision.models import resnet50

## Define Image Directories

Set the paths to the cat and dog image directories.

In [2]:
# Paths to your cat and dog images
cat_dir = "./data/cats_set"
dog_dir = "./data/dogs_set"

## Define Custom Dataset Class

Create a custom PyTorch Dataset class for loading and transforming images.

In [3]:
class ImageDataset(Dataset):
    """
    Custom Dataset for loading images.
    
    Attributes:
        image_dir (str): The directory containing the images.
        image_names (list): The list of image names.
        label (int): The label associated with the images.
        transform (callable): The transformation to be applied to the
    """

    def __init__(self, image_dir: str, label: int, transform: Compose=None):
        """
        Constructor method.

        Args:
            image_dir (str): The directory containing the images.
            label (int): The label associated with the images.
            transform (Compose): The transformation to be applied to the images.
        """
        self.image_dir = image_dir
        self.image_names = os.listdir(image_dir)
        self.label = label
        self.transform = transform
    
    def __len__(self):
        """
        Return the number of images in the dataset.
        
        Returns:
            int: The number of images in the dataset.
        """
        return len(self.image_names)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        """
        Return the image and its label at the given index.

        Args:
            idx (int): The index of the image to be returned.

        Returns:
            Tuple[torch.Tensor, int]: The image and its label.
        """
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.label

## Define Image Transformations

Create preprocessing transformations for ResNet50 (resize, normalize, etc.).

In [4]:
# Preprocess transformations.
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## Create Datasets and DataLoader

Create datasets for cats and dogs, combine them, and create a DataLoader.

In [5]:
# Create datasets and loaders for both classes.
cat_dataset = ImageDataset(cat_dir, label=0, transform=transform)
dog_dataset = ImageDataset(dog_dir, label=1, transform=transform)
combined_dataset = cat_dataset + dog_dataset  # Combine both datasets.
loader = DataLoader(combined_dataset, batch_size=32, shuffle=False)

## Load Pretrained ResNet50 Model

Load the pretrained ResNet50 model and remove the final classification layer to extract features.

In [6]:
# Load pretrained ResNet50 model.
model = resnet50(pretrained=True)

# Remove the final classification layer.
model = torch.nn.Sequential(*list(model.children())[:-1])

# Set model to evaluation mode.
model.eval()

/Users/fabricio/.miniforge3/envs/damin2025/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fabricio/.miniforge3/envs/damin2025/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


## Extract Features

Extract deep learning features from all images using the pretrained model.

In [7]:
# Extract features.
features = []
labels = []
with torch.no_grad():
    for imgs, lbls in loader:
        outputs = model(imgs)
        outputs = outputs.view(outputs.size(0), -1)
        features.append(outputs.cpu().numpy())
        labels.extend(lbls.numpy())

features = np.vstack(features)
labels = np.array(labels)

print(f"Extracted features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")

Extracted features shape: (1000, 2048)
Labels shape: (1000,)


## Save Features and Labels

Save the extracted features and labels to .npy files for later use in dimensionality reduction analysis.

In [8]:
# Save features and labels to .npy files.
np.save("./data/features.npy", features)
np.save("./data/labels.npy", labels)

print("Features and labels saved successfully!")
print(f"Features saved to: ./data/features.npy")
print(f"Labels saved to: ./data/labels.npy")

Features and labels saved successfully!
Features saved to: ./data/features.npy
Labels saved to: ./data/labels.npy
